### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2022
quarter = 2

today = date.today()
today

datetime.date(2022, 7, 27)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2022 AND quarter <= 2) 
OR (year = 2022-1 AND quarter >= 2+1)
ORDER BY year DESC, quarter DESC


In [3]:
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp.shape

(22, 5)

In [4]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(226, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct.sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
11,KTB,2022,Q2,27137553,18099033,9038520,49.94
21,TTB,2022,Q2,11791212,8169846,3621366,44.33
10,KKP,2022,Q2,7589746,5271949,2317797,43.96
2,BBL,2022,Q2,27306314,19694952,7611362,38.65
5,DELTA,2022,Q2,10330437,7635796,2694641,35.29


In [6]:
sql = """
DELETE FROM yr_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year, quarter)
rp = conlt.execute(sql)
rp.rowcount

20

In [7]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
rcds = df_ins.values.tolist()
len(rcds)

22

In [8]:
for rcd in rcds:
    print(rcd)

['AOT', 2022, 'Q2', -16784073, -13747749, -3036324, -22.09, 24]
['ASP', 2022, 'Q2', 578771, 818663, -239892, -29.3, 40]
['BBL', 2022, 'Q2', 27306314, 19694952, 7611362, 38.65, 50]
['BFIT', 2022, 'Q2', 566031, 1137533, -571502, -50.24, 58]
['COTTO', 2022, 'Q2', 597648, 618161, -20513, -3.32, 710]
['DELTA', 2022, 'Q2', 10330437, 7635796, 2694641, 35.29, 138]
['DTAC', 2022, 'Q2', 2733414, 4069550, -1336136, -32.83, 144]
['GVREIT', 2022, 'Q2', 671535, 788226, -116691, -14.8, 654]
['HMPRO', 2022, 'Q2', 5676476, 5740561, -64085, -1.12, 208]
['KBANK', 2022, 'Q2', 40536483, 39457834, 1078649, 2.73, 245]
['KKP', 2022, 'Q2', 7589746, 5271949, 2317797, 43.96, 255]
['KTB', 2022, 'Q2', 27137553, 18099033, 9038520, 49.94, 258]
['KTC', 2022, 'Q2', 6205952, 5856336, 349616, 5.97, 259]
['LHFG', 2022, 'Q2', 1090666, 1765194, -674528, -38.21, 268]
['OISHI', 2022, 'Q2', 834166, 706204, 127962, 18.12, 338]
['SCB', 2022, 'Q2', 36939973, 28509171, 8430802, 29.57, 426]
['SCC', 2022, 'Q2', 33904936, 49838986, 

In [9]:
sql = """INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)

INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [10]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [11]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [12]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
11,KTB,2022,Q2,27137553,18099033,9038520,49.94
21,TTB,2022,Q2,11791212,8169846,3621366,44.33
10,KKP,2022,Q2,7589746,5271949,2317797,43.96
2,BBL,2022,Q2,27306314,19694952,7611362,38.65
5,DELTA,2022,Q2,10330437,7635796,2694641,35.29


In [13]:
criteria_2 = df_ins.q_amt_p > 300000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
11,KTB,2022,Q2,27137553,18099033,9038520,49.94
21,TTB,2022,Q2,11791212,8169846,3621366,44.33
10,KKP,2022,Q2,7589746,5271949,2317797,43.96
2,BBL,2022,Q2,27306314,19694952,7611362,38.65
5,DELTA,2022,Q2,10330437,7635796,2694641,35.29


In [14]:
criteria_3 = df_ins.Pct > 5.0
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
11,KTB,2022,Q2,27137553,18099033,9038520,49.94
21,TTB,2022,Q2,11791212,8169846,3621366,44.33
10,KKP,2022,Q2,7589746,5271949,2317797,43.96
2,BBL,2022,Q2,27306314,19694952,7611362,38.65
5,DELTA,2022,Q2,10330437,7635796,2694641,35.29


In [15]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
11,KTB,2022,Q2,27137553,18099033,9038520,49.94
21,TTB,2022,Q2,11791212,8169846,3621366,44.33
10,KKP,2022,Q2,7589746,5271949,2317797,43.96
2,BBL,2022,Q2,27306314,19694952,7611362,38.65
5,DELTA,2022,Q2,10330437,7635796,2694641,35.29
15,SCB,2022,Q2,36939973,28509171,8430802,29.57
14,OISHI,2022,Q2,834166,706204,127962,18.12
19,TFFIF,2022,Q2,1209444,1125581,83863,7.45
12,KTC,2022,Q2,6205952,5856336,349616,5.97


In [16]:
df_ins.loc[df_ins_criteria,cols].sort_values(['name'],ascending=[True])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
2,BBL,2022,Q2,27306314,19694952,7611362,38.65
5,DELTA,2022,Q2,10330437,7635796,2694641,35.29
10,KKP,2022,Q2,7589746,5271949,2317797,43.96
11,KTB,2022,Q2,27137553,18099033,9038520,49.94
12,KTC,2022,Q2,6205952,5856336,349616,5.97
14,OISHI,2022,Q2,834166,706204,127962,18.12
15,SCB,2022,Q2,36939973,28509171,8430802,29.57
19,TFFIF,2022,Q2,1209444,1125581,83863,7.45
21,TTB,2022,Q2,11791212,8169846,3621366,44.33
